In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import spikeextractors as se
import spiketoolkit as st
import spikewidgets as sw
import time
import numpy as np
import matplotlib.pylab as plt

In [9]:
recording, sorting = se.example_datasets.toy_example1(num_channels=4, duration=30)

In [10]:
sorting_ms4 = st.sorters.mountainsort4(recording=recording, adjacency_radius=10)

Using 6 workers.
Using tmpdir: /tmp/tmppunn8yev
Num. workers = 6
Preparing /tmp/tmppunn8yev/timeseries.hdf5...
Preparing neighborhood sorters (M=4, N=900000)...
Neighboorhood of channel 3 has 4 channels.
Neighboorhood of channel 0 has 4 channels.
Neighboorhood of channel 1 has 4 channels.
Detecting events on channel 2 (phase1)...
Neighboorhood of channel 2 has 4 channels.
Detecting events on channel 1 (phase1)...
Detecting events on channel 4 (phase1)...
Detecting events on channel 3 (phase1)...
Elapsed time for detect on neighborhood: 0:00:00.176103
Num events detected on channel 2 (phase1): 643
Computing PCA features for channel 2 (phase1)...
Elapsed time for detect on neighborhood: 0:00:00.168457
Num events detected on channel 1 (phase1): 704
Computing PCA features for channel 1 (phase1)...
Elapsed time for detect on neighborhood: 0:00:00.173672
Num events detected on channel 4 (phase1): 1032
Computing PCA features for channel 4 (phase1)...
Elapsed time for detect on neighborhood: 0

In [11]:
print([len(sorting.getUnitSpikeTrain(i)) for i in sorting.getUnitIds()])

[73, 71, 72, 70, 68, 73, 73, 71, 70, 69]


In [12]:
sorting_ms4.getUnitIds()

[3]

In [13]:
rec_f = st.preprocessing.bandpass_filter(recording)

In [21]:
ts = time.time()
wf = st.postprocessing.getUnitWaveforms(rec_f, sorting_ms4, verbose=True)
print(time.time()-ts)

[3]
Waveform 1/1
0.003336668014526367


In [26]:
sorting_ms4.getUnitSpikeFeatureNames()

['waveforms']

In [27]:
templates = st.postprocessing.getUnitTemplate(recording, sorting_ms4)

Using 'waveforms' features


In [ ]:
sorting_ms4.getUnitPropertyNames()

In [ ]:
max_chan = st.postprocessing.getUnitMaxChannel(recording, sorting_ms4)

In [ ]:
sorting.getUnitPropertyNames()

In [ ]:
pca_scores = st.postprocessing.computePCAScores(recording, sorting_ms4)

In [ ]:
snr = st.postprocessing.computeUnitSNR(recording, sorting)

In [ ]:
print(sorting.getUnitPropertyNames())
print(sorting.getUnitSpikeFeatureNames())